In [7]:
import pandas as pd

In [2]:
df = pd.read_csv('data/processed_rybki_final.csv')

#### Feature Engineering 
Add informative features

In [3]:
# add general type of fish -gatunek
df['first_name'] = df['name'].apply(lambda name: name.split())
df['first_name'] = df['first_name'].apply(lambda list: list[0])

In [4]:
## add features for: how much food needed, separated food kinds
def cost_of_food(food_list, sep):
    if sep == "+":
        return len(food_list)
    else: return 1

df["food"] = df["food"].apply(lambda str: str.translate({ord(i): None for i in '[],\"\''}))
df["food"] = df["food"].apply(lambda str: str.split())

f_list = ["food_1","food_2","food_3"]

for idx, val in enumerate(f_list):
    df[val] = df["food"].apply(lambda l: 0 if len(l) < idx + 1 else l[idx])

df["food_needed"] = df.apply(lambda row: cost_of_food(row["food"], row["food_separator"]), axis = 1)

### One-hot encoding


In [5]:
train_df = df[['name', 'latin_name', 'temperature', 'length', 'location', 'kryjowka_info', 'breeding_info', 'Lowest_temperature',
       'Top_temperature', 'first_name', 'food_1', 'food_2', 'food_3', 'food_needed']]

f1 = pd.get_dummies(df['food_1'],dtype = float, prefix="")
f2 = pd.get_dummies(df['food_2'],dtype = float, prefix="")
f3 = pd.get_dummies(df['food_3'],dtype = float, prefix="")

In [6]:
f = pd.concat([f1,f2,f3]).groupby(level=0).any().astype(int)
f = f.drop("_0", axis=1)

rybki_names = train_df["name"]
train_df = pd.concat([train_df,f], axis= 1)
train_df = train_df.drop(["food_1","food_2","food_3","latin_name","name"],axis=1)
train_df["location"] = train_df["location"].apply(lambda x: x if x != "Nowa Gwinea" else "Australia")
train_df["location"] = train_df["location"].apply(lambda x: x if x not in["Tanganika", "Malawi"] else "Afryka")

train_df = pd.get_dummies(train_df,dtype = float,columns=["kryjowka_info","breeding_info","first_name","temperature","location"], prefix=["kryjowka","","gatunek","","location"])
train_df.to_csv("data/train_data.csv")

In [8]:
len(train_df.columns)


## to reduce number of features if necessary:
# wyrzucic jedzenie oprocz random, 

114

#### Determining number of clusters

In [9]:
from sklearn.cluster import KMeans 



In [10]:
kmeans = KMeans(n_clusters=7, random_state=0, n_init="auto").fit(train_df)
kmeans.labels_

array([1, 5, 2, 2, 5, 5, 3, 1, 1, 1, 1, 1, 1, 1, 5, 1, 5, 2, 3, 5, 5, 1,
       3, 3, 3, 5, 3, 1, 5, 1, 1, 5, 5, 1, 5, 5, 3, 5, 6, 1, 1, 1, 1, 6,
       3, 1, 1, 1, 3, 3, 2, 1, 5, 1, 3, 6, 3, 2, 2, 1, 3, 1, 3, 3, 6, 2,
       1, 1, 5, 1, 1, 1, 5, 5, 3, 3, 1, 3, 5, 1, 3, 3, 1, 1, 5, 2, 5, 3,
       5, 1, 1, 3, 3, 1, 1, 3, 1, 1, 1, 1, 3, 5, 1, 1, 3, 5, 4, 5, 2, 1,
       1, 3, 1, 5, 0, 3, 0, 3, 1, 1, 1, 3, 5, 5, 3, 5, 1, 5, 3, 5, 5, 5,
       2, 3, 3, 1, 1, 2, 1, 1, 1, 1, 1, 5, 5, 5, 1, 1, 5, 1, 1, 1, 1, 1,
       1, 1, 1, 5, 2, 1, 2, 6, 1, 1, 6, 2, 0, 5, 5, 6, 5, 5, 1, 5, 1, 6,
       1, 1, 1, 1, 6, 5, 5, 6, 6, 6, 6, 5, 1, 3, 3, 1, 5, 1, 5, 6, 6, 5,
       5, 1, 1, 6, 1, 6, 5, 1, 5, 6, 3], dtype=int32)

In [11]:
train_df["class"] = kmeans.labels_
train_df["name"] = rybki_names

In [12]:
train_df

,length,Lowest_temperature,Top_temperature,food_needed,_glony,_mrożony,_random,_suchy,_ślimaki,_żywy,...,_images/zimno-ciepla.png,_images/zimno-srednia.png,location_Afryka,location_Ameryka Płd.,location_Ameryka Płn.,location_Ameryka Śr.,location_Australia,location_Azja,class,name
0,4.0,22,30,2,0,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,Gupik Endlera
1,10.0,22,28,1,0,1,0,1,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,Giętkoząb czarnobrzuchy
2,20.0,23,28,1,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,Glonojad syjamski
3,20.0,22,30,1,0,0,0,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,Gurami całujący
4,15.0,22,26,3,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5,Grubowarg dwubarwny – Labeo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,13.0,24,28,2,1,0,0,0,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,Pyszczak wydłużony
205,8.0,26,28,1,0,0,0,0,1,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,Pyszczak saulosi
206,10.0,24,28,1,0,0,0,1,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5,Pyszczak Yellow – Żółty
207,30.0,24,26,1,0,0,0,0,0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6,Pirania czerwona – Natterera


In [13]:
train_df.groupby(['class']).size()

class
0     3
1    84
2    14
3    37
4     1
5    52
6    18
dtype: int64

In [15]:
train_df.loc[train_df["class"] == 4]

,length,Lowest_temperature,Top_temperature,food_needed,_glony,_mrożony,_random,_suchy,_ślimaki,_żywy,...,_images/zimno-ciepla.png,_images/zimno-srednia.png,location_Afryka,location_Ameryka Płd.,location_Ameryka Płn.,location_Ameryka Śr.,location_Australia,location_Azja,class,name
106,90.0,22,26,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,Sum rekini


In [1]:
import pyreadr

ptaki_df = pyreadr.read_r('birds.rda')

In [9]:
ptaki_df

OrderedDict([('birds',
                               common_name                scientific_name       set  \
              0             Abbott's Booby               Papasula abbotti   oceania   
              1           Acorn Woodpecker        Melanerpes formicivorus      core   
              2            American Avocet        Recurvirostra americana      core   
              3           American Bittern          Botaurus lentiginosus      core   
              4              American Coot               Fulica americana      core   
              ..                       ...                            ...       ...   
              352     Yellow-Billed Cuckoo            Coccyzus americanus      core   
              353     Yellow-Breasted Chat                 Icteria virens      core   
              354  Yellow-Headed Blackbird  Xanthocephalus xanthocephalus      core   
              355    Yellow-Rumped Warbler             Setophaga coronata      core   
              356   